# kotlin generics
https://kotlinlang.org/docs/generics.html

kotlinのgenericsを覚えるノート。
写経しつつ、自分の言葉で書いてみる。

* Boxは「型パラメータ」を必要とするクラス
* `Box<Int>` で型を指定する

In [2]:
class Box<T>(t: T) {
    var value = t
}

In [3]:
val box: Box<Int> = Box<Int>(1)

In [4]:
println(box.value)

1


コンストラクタ引数にパラメータが推論可能な場合は、型パラメータを省略できる（＝型推論）

In [6]:
val box2 = Box(2)

In [7]:
println(box2.value)

2


## 変位
ここから難しいやつ。
Javaのジェネリクスは不変。つまり、`List<String>`は `List<Object>` のサブタイプではない。
`List`が不変でない場合、次のコードはコンパイルされるが、実行時に例外が発生する。

```java
// Java
List<String> strs = new ArrayList<String>();
List<Object> objs = strs; // 本当はコンパイルエラーになるけど、不変でない場合はコンパイルできる
objs.add(1);
String s = strgs.get(0); // ランタイムエラー、ClassCastExceptionになる。
```

↑のようなランタイムエラーを起きないようにするために、コンパイルエラーで防いでいる。
けど、別の影響もある。たとえば、`Collection` interfaceの `addAll` methodを考えてみよう。

直感的に書くとシグネチャは次になるかもしれない。
```java
// Java
interface Collection<E> ... {
    void addAll(Collection<E> items);
}
```

けれども実際は次。

```java
// Java
interface Collection<E> ... {
    void addAll(Collection<? extends E> items);
}
```

ワイルドカード型引数 `? exnteds E`は、E自体とEのサブタイプを受け入れる。
型Eの要素を読み取りできる。けれども、オブジェクトのクラスはどのクラスかは分からないので、書き込みはできない
（Eを継承しているのは知ってるけど、どのEのサブタイプかまでは知らない）。

この制限と引き換えに、望む振る舞いは `Collection<String>` は  `Collection<? exnteds Object>` になること（？）
言い換えると、 exnteds付きのワイルドカード(upper-bound)は型を共変にします。

これが機能する理由が簡単。
Collectionからのみ値を取得できる場合は、`Collection<String>`を使ってObjectを読み出しすることは問題ない。
逆に、Collectionへ値をputするだけの場合は、Collection<Object>`を使ってStringをputできる。

Javaの場合、`List<? super String>` で、 `List<Object>` のスーパータイプ。
後者は「反変性」と呼び、`List<? super String>`に対してStringを取るメソッドを呼ぶことができる。
（たとえば、 `add(String)` とか `set(int, String)`とか）

`List<T>`でTを返すメソッドを呼ぶと、Stringは得られなくて、Objectが得られる。

読み取り専用のオブジェクトは「プロデューサー」、書き込み専用のオブジェクトは「コンシュマー」。


> If you use a producer-object, say, List<? extends Foo>, you are not allowed to call add() or set() on this object, but this does not mean that it is immutable: for example, nothing prevents you from calling clear() to remove all the items from the list, since clear() does not take any parameters at all. 
The only thing guaranteed by wildcards (or other types of variance) is type safety. Immutability is a completely different story.y.

### 定義元の変位
`Source<T>` に引数を取らなくて、`T`を返却するメソッドを用意してみる

```java
// Java
interface Source<T> {
    T nextT();
}
```

この場合、`Source<Strings>` の参照を `Source<Object>`に入れても安全ははず。
だって、コンシュマーな呼び出し（書き込み処理）がないのだから。
けれどもJavaはこのことを知らないから、コンパイルエラーになる。

```java
// Java
void demo(Source<String> strs) {
    Source<Object> objects = strs; // コンパイルエラー
}
```

この問題を修正するために、`Source<? extends Object>` と型パラメータを宣言する必要がある。

> Doing so is meaningless, because you can call all the same methods on such a variable as before, so there's no value added by the more complex type. But the compiler does not know that.

（↑解説っぽい文章。Javaだと難しい理由が書いてある）

> In Kotlin, there is a way to explain this sort of thing to the compiler. This is called declaration-site variance: you can annotate the type parameter T of Source to make sure that it is only returned (produced) from members of Source<T>, and never consumed. To do this, use the out modifier:

kotlinの場合、コンパイラーに情報を渡せる。その情報を「定義元の変位」と呼んでいる。
型パラメータTに注釈をつけて、それが `Source<T>`のメンバーから読み返却され(produced)、
消費されないようにできる。

↑のようにするには、`out` をつける


.

In [12]:
// OK
interface Source<out T> {
    fun nextT(): T
}
fun demo(strs: Source<String>) {
    val objects: Source<Any> = strs
}

In [18]:
// NG: outが付かない場合はコンパイルエラーになる
interface Source<T> {
    fun nextT(): T
}
fun demo(strs: Source<String>) {
    val objects: Source<Any> = strs
}

Line_17.jupyter.kts (6:9 - 16) Variable 'objects' is never used
Line_17.jupyter.kts (6:32 - 36) Type mismatch: inferred type is Line_17_jupyter.Source<String> but Line_17_jupyter.Source<Any> was expected

一般的なルールとして、


`class C<out T>`と宣言する場合、Cのメンバーの外側で発生するけど（？）、その代わりに`C<Base>`は安全にサブタイプである`C<Derived>`になれる。

言い換えれば、Cの型パラメータTは共変、Tは共変な型パラメータ。
Cは、Tのproducerであり、Tのconsumerではない（＝Tを読み取りできる）。

`out` 修飾子を変位アノテーションと呼び、型パラメータ宣言場所で提供されるから、「宣言元の変位」と呼ぶ。

これは、型の使用におけるワイルドカードによって型が共変になるJavaの「呼び出し元の変位」とは対象的（？）

`out` に加えて、kotlinは補完的な変位 `in` がある（対になってるのかな）。
型パラメータを反変にする。書き込みだけでき、読み取りはできない。

反変な型の良い例として、 `Comparable` がある。

In [16]:
// OK
interface Comparable<in T> {
    operator fun compareTo(other: T): Int
}

fun demo(x: Comparable<Number>) {
    x.compareTo(1.0) // 1.0はDouble. DoubleはNumberのサブタイプ
    // だから、Comparable<Double>にxを割り当てできる
    val y: Comparable<Double> = x // OK
}

In [19]:
// NG: inをつけないとコンパイルエラーになる
interface Comparable<T> {
    operator fun compareTo(other: T): Int
}

fun demo(x: Comparable<Number>) {
    x.compareTo(1.0) // 1.0はDouble. DoubleはNumberのサブタイプ
    // だから、Comparable<Double>にxを割り当てできる
    val y: Comparable<Double> = x // OK
}

Line_18.jupyter.kts (9:9 - 10) Variable 'y' is never used
Line_18.jupyter.kts (9:33 - 34) Type mismatch: inferred type is Line_18_jupyter.Comparable<Number> but Line_18_jupyter.Comparable<Double> was expected

C#で長い間使われていたキーワードだから、簡単に覚えれるでしょ。

## 型投影

### 用語
* variance: 変位
* invariant: 不変
* covariant: 共変
* contravariance: 反変性
* subtype: あるclass Aを継承したclass Bのこと
* supertype: あるclass Bの継承元の、あるclass Aのこと
* signature: return type, method name, method argumentsから構成するもの。メソッドを識別するのに使う。
* call-site/use-site: 呼び出し元。定義した関数を呼び出す場所。
* declaration-site: 定義した場所